
# This is a test


In [ ]:
IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'f' ) 
BEGIN
	EXEC sp_executesql N'CREATE SCHEMA f AUTHORIZATION dbo;'
END
;

GO

/*********************************************************/
/******************  Customer DIM DDL   ******************/
/*********************************************************/

IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'Customers')
BEGIN
	CREATE TABLE dim.Customers(
	pkCustomer int IDENTITY(1000,1) NOT NULL,
	CustomerID nvarchar(5) NOT NULL,
	Customer nvarchar(40) NOT NULL,
	City nvarchar(15) NULL,
	Country nvarchar(15) NULL,
	LoadDate DATE NOT NULL,
	SourceCountry nvarchar(25) NULL
	)
	;

	ALTER TABLE dim.Customers
	ADD CONSTRAINT PK_Customers_LUP PRIMARY KEY(pkCustomer);

	ALTER TABLE dim.Customers
    ADD CONSTRAINT UC_Customers_ID UNIQUE (CustomerID);

END


GO

/*********************************************************/
/****************** Calendar DIM Script ******************/
/*********************************************************/

IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'Calendar')
BEGIN
-- Create the Calendar table
CREATE TABLE dim.Calendar
(
    pkCalendar INT NOT NULL,
    DateValue DATE NOT NULL,
    Year INT NOT NULL,
    Quarter INT NOT NULL,
	Qtr VARCHAR(3) NOT NULL,
    Month INT NOT NULL,
    MonthName VARCHAR(10) NOT NULL,
	MonthShort VARCHAR(3) NOT NULL,
    Week INT NOT NULL,
    Day INT NOT NULL,
	DayName VARCHAR(10) NOT NULL,
	DayShort VARCHAR(3) NOT NULL,
    IsWeekday BIT NOT NULL,
	Weekday VARCHAR(3) NOT NULL
);

	ALTER TABLE dim.Calendar
	ADD CONSTRAINT PK_Calendar_Julian PRIMARY KEY(pkCalendar);

	ALTER TABLE dim.Calendar
    ADD CONSTRAINT UC_Calendar UNIQUE (DateValue);
END

GO

/*********************************************************/
/******************  Shipper DIM DDL    ******************/
/*********************************************************/
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'Shipper')
BEGIN
-- Create the Calendar table
CREATE TABLE dim.Shipper
(
      pkShipID INT NOT NULL
	, Shipper NVARCHAR(40) NOT NULL
	, Phone NVARCHAR(24) NULL
);

	ALTER TABLE dim.Shipper
	ADD CONSTRAINT PK_Ship PRIMARY KEY(pkShipID);
END

GO

/*********************************************************/
/******************  Products DIM DDL   ******************/
/*********************************************************/

IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'Products')
BEGIN
CREATE TABLE dim.Products(
	  pkProdID INT NOT NULL
	, ProductID INT NOT NULL
	, Product NVARCHAR(50) NOT NULL
	, ProductCategory NVARCHAR(25) NOT NULL
	, CategoryDesc NTEXT NULL
	, UnitPrice MONEY NULL
	, UnitsInStock INT NULL
	, UnitsOnOrder INT NULL
	, ReorderLevel INT NULL
	, Discontinued BIT NOT NULL
	, ReorderFlag NVARCHAR(3) NOT NULL
	);

	ALTER TABLE dim.Products
	ADD CONSTRAINT PK_Prods PRIMARY KEY(pkProdId);

	ALTER TABLE dim.Products
    ADD CONSTRAINT UC_Prods UNIQUE (ProductId);
END
;
GO



/*********************************************************/
/******************  Employees DIM DDL   ******************/
/*********************************************************/
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'Employees')
BEGIN
CREATE TABLE dim.Employees(
	EmployeeID INT NOT NULL,
	Employee NVARCHAR(50) NOT NULL,
	Title NVARCHAR(30) NULL,
	BirthDate DATETIME NULL,
	HireDate DATETIME NULL,
	City NVARCHAR(15) NULL,
	Country NVARCHAR(15) NULL,
	ReportsTo INT NULL
);
	ALTER TABLE dim.Employees
	ADD CONSTRAINT PK_Emp PRIMARY KEY(EmployeeID);

END

GO




/*********************************************************/
/*********************************************************/
/*********************************************************/
/******************  Fact Table Builds  ******************/
/*********************************************************/
/*********************************************************/
/*********************************************************/



/*********************************************************/
/******************  OrderPerf f.Table  ******************/
/*********************************************************/

-- OrderID, OrderDate, Customer, Freight(f)
IF EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'f' AND TABLE_NAME = 'OrderPerf')
BEGIN 
	DROP TABLE f.OrderPerf;
END

GO

CREATE TABLE f.OrderPerf(
	  OrderID INT NOT NULL
	, fkCalendar INT NOT NULL
	, fkCustomer INT NOT NULL
	, fkShipper INT NULL
	, fkEmployee INT NULL
	, Freight MONEY NOT NULL
	, DaysToShip INT NULL
	, DaysTilRequired INT NULL
	, OrderCount INT NOT NULL
);

-- Could use PRIMARY or UNIQUE for the ORDERID - just used to not load dupes

ALTER TABLE f.OrderPerf
ADD CONSTRAINT PK_ORD PRIMARY KEY(OrderID)
;

ALTER TABLE f.OrderPerf
ADD CONSTRAINT FK_ORDtoCAL
	FOREIGN KEY (fkCalendar)              -- FROM the LOCAL TABLE
	 REFERENCES  dim.Calendar(pkCalendar) -- TO the FOREIGN TABLE
;

ALTER TABLE f.OrderPerf
ADD CONSTRAINT FK_ORDtoCUST
	FOREIGN KEY (fkCustomer)              -- FROM the LOCAL TABLE
	 REFERENCES  dim.Customers(pkCustomer) -- TO the FOREIGN TABLE
;

ALTER TABLE f.OrderPerf
ADD CONSTRAINT FK_ORDtoSHIP
	FOREIGN KEY (fkShipper)
	 REFERENCES dim.Shipper(pkShipId)
;

ALTER TABLE f.OrderPerf
ADD CONSTRAINT FK_ORDtoEMP
	FOREIGN KEY (fkEmployee)
	 REFERENCES dim.Employees(EmployeeID)
;

/*********************************************************/
/******************  ProductsPerf f.Table  ******************/
/*********************************************************/

IF EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'f' AND TABLE_NAME = 'ProductPerf')
BEGIN 
	DROP TABLE f.ProductPerf;
END

CREATE TABLE f.ProductPerf(
	OrderID INT NOT NULL,
	OrderDate INT NOT NULL,
	ProductID INT NOT NULL,
	CustomerID INT NOT NULL,
	UnitPrice MONEY NOT NULL,
	Quantity SMALLINT NOT NULL,
	Discount FLOAT NOT NULL,
	DiscFlag INT NOT NULL,
	LineTotal MONEY NULL
);

ALTER TABLE f.ProductPerf
ADD CONSTRAINT PK_ProdPerf PRIMARY KEY(OrderID, ProductID)

ALTER TABLE f.ProductPerf
ADD CONSTRAINT FK_PRODtoCAL
	FOREIGN KEY (OrderDate)
	 REFERENCES dim.Calendar(pkCalendar)
;

ALTER TABLE f.ProductPerf
ADD CONSTRAINT FK_PRODtoPROD
	FOREIGN KEY (ProductID)
	 REFERENCES dim.Products(pkProdId)
;

ALTER TABLE f.ProductPerf
ADD CONSTRAINT FK_PRODtoCUST
	FOREIGN KEY (CustomerID)
	 REFERENCES dim.Customers(pkCustomer)
